# Section 8. Clustering

#### Instructor: Pierre Biscaye 

The content of this notebook is taken from UC Berkeley D-Lab's Python Machine Learning [course](https://github.com/dlab-berkeley/Python-Machine-Learning).

### Sections
1. K-Means clustering, evaluation, interpretation, limitations
2. Hierarchical clustering, agglomerative clustering

**Clustering** is an unsupervised ML method used to group data points based on their features alone, with no observed grouping labels as in supervised classification. Thus most clustering alorithms seeks to group points by their distance in a high dimensional space generated by provided features.

Below is a plot showing the results of the clustering algorithms in Scikit-Learn for several different toy datasets.

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png'/>

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs, make_circles, make_moons

#plt.style.use("ggplot")

## Data and Clustering Objective

[Spotify has a really cool api](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features) that provides access to a variety of numerical features encoding musical traits such as energy, danceability, and loudness. Below is the data dictionary explaining what each feature means.

### Spotify Data Dictionary

**acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

**danceability:** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

**duration_ms:** The duration of the track in milliseconds.

**energy:** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

**instrumentalness:** Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

**key:** The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.

**liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.

**loudness:** The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.

**mode:** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

**speechiness:** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

**tempo:** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

**time_signature:** An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".

**valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

There are also song identifiers, including artist, album, and song names and internal Spotify IDs.

### Objective

Our job is to cluster around 25,000 songs based on the data provided by Spotify.

In [ ]:
df = pd.read_csv("data/spotify_features.csv")
df.head()

In [ ]:
df.info()

## 1. K-means clustering  

The first algorithm we cover is k-means clustering using `scikit-learn`. The scikit-learn documentation for clustering is found [here](http://scikit-learn.org/stable/modules/clustering.html).


![](https://miro.medium.com/max/1200/1*rw8IUza1dbffBhiA4i0GNQ.png)

K-Means works by dividing data into K number of group, where K is determined by you the data scientist.

The K-Means algorithm assigns points to a group based on distance to that group's centroid. The algorithm works to find the centroids that minimize the *inertia*, or the sum of the squared distances, of every point and the possible centroids as shown in the following formula:

$$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_j, \mu_i||^2)$$

Here $C$ is the set of centroids and the double bars represent a distance metric.

Distance is determined using the euclidean distance formula, as in this two-dimensional case:

$$d = \sqrt {\left( {x_1 - x_2 } \right)^2 + \left( {y_1 - y_2 } \right)^2 }$$

The algorithm first randomly comes up with the positions of the K centroids and then assigns labels to data based on their nearest centroids.

After this first iteration the centroids are then recomputed by taking the average of all the points in each cluster. This process repeats until the model reaches a point of convergance or some other stopping criteria.

The following gif demonstrates the iterative process of KMeans

<img src=https://upload.wikimedia.org/wikipedia/commons/e/ea/K-means_convergence.gif style="width: 500px;"/>

### Visualizing K-Means in a 2D space

Before we apply Kmeans to the spotify data, let's visualize how it works on a 2D matrix. We'll simulate data using `make_blobs` which generates synthetic datasets for clustering using Gaussian (following a normal distribution) 'blobs' of data points. 

In [ ]:
X, _ = make_blobs(n_samples=200, n_features=2, random_state=4)
X[:5]

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(x = X[:, 0], y = X[:, 1]);

It seems obvious there are two clusters, so let's intialize the algoritm with K = 2.

In [ ]:
km = KMeans(n_clusters=2, random_state=1)

Fit on the data

In [ ]:
km.fit(X)

Grab the labels and centroids. (FYI, `labels_` has an _ on the end because it's an attribute created after a certain command, there is no `labels_` pre modeling fitting).

In [ ]:
labels = km.labels_
labels

In [ ]:
# We can also grab labels predicting X
labels2 = km.predict(X) 
labels2

These are the centroids/centers of the clusters. They are the average values of the features in each cluster.

In [ ]:
centroids = km.cluster_centers_
centroids

Let's plot the data colored by the cluster labels along with the centriods.

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(x = X[:, 0], y = X[:, 1], c=labels, cmap="autumn")
plt.scatter(x = centroids[:, 0], y = centroids[:, 1], 
            s=800, marker="*", c=list(set(labels)), 
            edgecolors=["black", "black"], cmap="autumn", linewidths=3);

What happens if we try a different K value?

In [ ]:
km = KMeans(n_clusters=3, random_state=1)
km.fit(X)
labels = km.labels_
centroids = km.cluster_centers_
plt.figure(figsize=(10, 8))
plt.scatter(x = X[:, 0], y = X[:, 1], c=labels, cmap="autumn")
plt.scatter(x = centroids[:, 0], y = centroids[:, 1], 
            s=800, marker="*", c=list(set(labels)), 
            edgecolors=["black","black", "black"], cmap="autumn", linewidths=3);

The algorithm will do its best to identify however many clusters you desire, but how much the addition of clusters adds to your understanding of the data depends upon its underlying structure. 

In [ ]:
km = KMeans(n_clusters=5, random_state=1)
km.fit(X)
labels = km.labels_
centroids = km.cluster_centers_
plt.figure(figsize=(10, 8))
plt.scatter(x = X[:, 0], y = X[:, 1], c=labels, cmap="autumn")
plt.scatter(x = centroids[:, 0], y = centroids[:, 1], 
            s=800, marker="*", c=list(set(labels)), 
            edgecolors=["black","black", "black"], cmap="autumn", linewidths=3);

## K-Means and Spotify

Now we'll apply K-Means clustering to the Spotify data. 

The first thing we will do is select the song attribute columns for clustering. This determines the dimensions over which cluster distances are calculated for identifying optimal centroids.

In [ ]:
df.head()

For simplicity and efficiency, let's select the mathematical measures of musical 'personality'.

In [ ]:
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness", "speechiness"]
X = df[cols]
X.head()

### Standardization

Before we cluster the data, we first need to standardize the features. Because the euclidean distance function equally considers or weights each feature, larger features bias the results.

We are going to transform the data into their Z-scores using the `StandardScaler` function.

In [ ]:
# Initialize scaler object
scaler = StandardScaler()
# Fit on the data
scaler.fit(X)

`scaler` has learned from the data but it hasn't produced any new data. That's why we need to use `.transform()`

In [ ]:
#Create the transformed data using scaler
Xs = scaler.transform(X)

We can also do both operations in one line, as we've done previously.

In [ ]:
Xs = scaler.fit_transform(X)

In [ ]:
Xs[0:5,]

Now we can cluster the spotify data. Let's try five clusters.

In [ ]:
km = KMeans(n_clusters=5, random_state=1)
km.fit(Xs)

It's not straightforward to map the data and clusters with so many features. But we can grab the labels and tally the frequency in each one.

In [ ]:
labels_5 = km.labels_
pd.value_counts(labels_5)

### Model Evaluation

Evaluating cluster models isn't as precise as evaluating supervised learning models since we don't know the true labels of the data.

However use we can use techniques such as the elbow method and silhouette scores to try to deterine the optimal k-value.

**The Elbow Method**
![](https://www.researchgate.net/publication/339823520/figure/fig3/AS:867521741733888@1583844709013/The-elbow-method-of-k-means.png)

The `intertia_` attribute tells us the model's error — the sum of the squared distances between every point and its centriod.

We make an 'elbow' plot by plotting an array of k-values versus each k-value's model's inertia score. An optimal number of clusters can be identified as an elbow or inflection point, where inertia is no longer decreasing as sharply with additional clusters.

In [ ]:
km.inertia_

In [ ]:
#Intialize list to collect intertia scores and array of k-values
i_scores = []
kvalues = np.arange(2, 11)

#iterate over kvalues
for k in kvalues:
    #intialize model with k and fit it on Xs
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(Xs)
    #append inertia score to i_scores
    i_scores.append(km.inertia_)
    
#plot kvalues versus i_scores
plt.figure(figsize=(10, 8))
plt.plot(kvalues, i_scores, linewidth = 3)
plt.xticks(ticks=kvalues, labels=kvalues)
plt.xlabel("K Values")
plt.ylabel("Inertia Scores");

**Question**: Based on this graph, where would you say the inflection or elbow point is?

**Silhouette Score**

![](https://uploads-ssl.webflow.com/5f5148a709e16c7d368ea080/5f7dea907b8e8c7769e769c8_5f7c9650bc3b1ed0ad2247eb_silhouette_formula.jpg)

**Question**: Do you understand why the different extreme values of $s(i)$ indicate different types of clusters?

Let's calculate silhouette scores across different numbers of clusters and compare.

In [ ]:
#Intialize list to collect silhouette scores
s_scores = []

#iterate over kvalues
for k in kvalues:
    #intialize model with k and fit it on Xs
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(Xs)
    labels = km.labels_
    #derive silhouette score by passing in data and labels
    ss = silhouette_score(Xs, labels=labels)
    #append silhouette score to s_scores
    s_scores.append(ss)
    
#plot kvalues versus s_scores
plt.figure(figsize=(10, 8))
plt.plot(kvalues, s_scores, linewidth = 3)
plt.xticks(ticks=kvalues, labels=kvalues)
plt.xlabel("K Values")
plt.ylabel("Silhouette Scores");

**Question**: What consistutes a good silhouette score? With that in mind, which k value produces the best silhouette score?

In [ ]:
#Find the maxiumum of s_scores and use that number to find its index value in s_scores
k_index = s_scores.index(max(s_scores))
#Index kvalues with k_index
best_k = kvalues[k_index]
best_k

That's not the $k$ we originally used! It is easy to re-train the model using the `best_k` value.

In [ ]:
km = KMeans(n_clusters=best_k, random_state=1)
km.fit(Xs)
labels = km.labels_
pd.value_counts(labels)

### Exploratory Data Analysis 

Let's stick with the original model's classification of 5 types of songs. We now need to add some identity to the labels. Sklearn doesn't tell us what each label means, the labels of 0, 1, 2, ... hold no information about the types of music each label represents.

Therefore we need to use some EDA techniques to derive meaning from these categorizations.

First, we'll group the original dataset by the labels and calculate the means. These are the cluster centroids.

In [ ]:
X["labels"] = labels_5

In [ ]:
pd.value_counts(labels_5)

In [ ]:
X.groupby('labels').mean()

What does this suggest about what the 5 labels describe?

Let's visualize the distributions of features by cluster.

In [ ]:
f, axes = plt.subplots(4, 2, figsize=(16, 15))

palette = "bright"
sb.histplot(data = X, x = 'acousticness', hue="labels", kde = True , palette = palette, ax=axes[0,0])
sb.histplot(data = X, x = 'danceability', hue="labels", kde = True , palette = palette, ax=axes[0,1])
sb.histplot(data = X, x = 'energy', hue="labels", kde = True , palette = palette, ax=axes[1,0])
sb.histplot(data = X, x = 'instrumentalness', hue="labels", kde = True , palette = palette, ax=axes[1,1])
sb.histplot(data = X, x = 'liveness', hue="labels", kde = True , palette = palette, ax=axes[2,0])
sb.histplot(data = X, x = 'loudness', hue="labels", kde = True , palette = palette, ax=axes[2,1])
sb.histplot(data = X, x = 'speechiness', hue="labels", kde = True , palette = palette, ax=axes[3,0]);

**Question**: What insights can we glean from this visualization? What are some other ways we can add meaning to the labels?

This is a challenge for *unsupervised* machine learning. We need to interpret and make sense of the classifications that result from the model, to try and determine what is underlying them. This is harder with more features than with fewer.

### The limits of K-Means

The biggest flaw of K-Means is its inability to coherently cluster complicated and irregular patterns of data.

Let's look at some examples.

In [ ]:
#Generate the circles moons datasets
circles, _ = make_circles(n_samples=200, factor=.3, noise = .04, random_state=1)
moons, _ = make_moons(n_samples=200, noise = .07, random_state=1)

In [ ]:
#Visualize circles dataset
plt.scatter(*circles.T);

In [ ]:
#Visualize moons dataset
plt.scatter(*moons.T);

**Question** How would you go about clustering these two datasets? How do you think K-Means will cluster them?

Let's use K-Means to identify two clusters for each of the two datasets and then visualize the results.

In [ ]:
km = KMeans(n_clusters=2, random_state=1)
km.fit(circles)
labels = km.labels_
centroids = km.cluster_centers_

plt.figure(figsize=(10, 8))
plt.scatter(x = circles[:, 0], y = circles[:, 1], c=labels, cmap="autumn")
plt.scatter(x = centroids[:, 0], y = centroids[:, 1], 
            s=800, marker="*", c=list(set(labels)), 
            edgecolors=["black", "black"], cmap="autumn", linewidths=3);

In [ ]:
km = KMeans(n_clusters=2, random_state=1)
km.fit(moons)
labels = km.labels_
centroids = km.cluster_centers_

plt.figure(figsize=(10, 8))
plt.scatter(x = moons[:, 0], y = moons[:, 1], c=labels, cmap="autumn")
plt.scatter(x = centroids[:, 0], y = centroids[:, 1], 
            s=800, marker="*", c=list(set(labels)), 
            edgecolors=["black", "black"], cmap="autumn", linewidths=3);

## 2. Hierarchical Clustering

Now we'll discuss an alternative clustering method: hierarchical clustering. In particular, we'll look at using agglomerative clustering. The documentation is [here](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering) in case you want to know more about the parameters.

Agglomerative clustering is a *bottom-up approach* that works by first assigning every data point its own cluster and then iteratively merging the closest clusters. This process involves first using a defined distance metric (euclidean, cosine, etc...) it measures the distance for every combination of clusters, and then merging the two clusters that are closest to each other. At the next iteration, the distances are recalculated, and a new merge occurs. The process continues until all points are in the same cluster or a stopping condition such as a cluster distance or a predefined number of clusters is met.

Hierarchical clustering is best explained through the graphic below called a dendrogram.

### Dendogram
![](https://miro.medium.com/max/740/1*VvOVxdBb74IOxxF2RmthCQ.png)


All the datapoints are laid out on the x-axis and the y-axis represents the distance threshold. The distance threshold has a negative correlation with the number of clusters.

The number of clusters in the figure above is four, which you can tell by the number of lines the red dotted-line crosses. The datapoints of f, g, and h are grouped in the same cluster because all of that cluster's intra-distances are less than the chosen distance threshold.

### Linkage 

Linkage is hierarchical clustering's method of determining distances between clusters for when it either splits up or combines clusters depending on the movement of the distance threshold. Different linkage types determine what distances are calculated.

![](https://editor.analyticsvidhya.com/uploads/40351linkages.PNG)

Single distance is effectively minimum distance while complete linkage is maximum difference. The former tends to create long chain-like clusters while the latter creates compact spherical clusters. Average linkage balances the compactness of clusters and sentitivity to outliers.

An interesting and useful way to think about hierarchical clustering is to approach the same way you would a biological taxonomy.

![](https://i.pinimg.com/originals/da/70/81/da708128987034e6c0b3b8a0ccac3c05.jpg)

The looser the taxonomical criteria the larger and more diverse collection of animals you get. For example: housecats, bobcats, lynx, tigers, lions, and leopards are a part of the felidae family despite possessing some obvious differences among them.

### Clustering with Agglomerative Clustering

Let's see if Agglomerative Clustering can do a better job with the moons and circles datasets than K-Means, while demonstrating the impact of different linkage inputs.

In [ ]:
#Intialize model with linkage = single
agg = AgglomerativeClustering(n_clusters=2, linkage="single")
#Fit on circles dataset
agg.fit(circles)
labels = agg.labels_

plt.figure(figsize=(10, 8))
plt.scatter(x = circles[:, 0], y = circles[:, 1], c=labels, cmap="autumn");

In [ ]:
#Intialize model with linkage = complete
agg = AgglomerativeClustering(n_clusters=2, linkage="complete")
#Fit on circles dataset
agg.fit(circles)
labels = agg.labels_

plt.figure(figsize=(10, 8))
plt.scatter(x = circles[:, 0], y = circles[:, 1], c=labels, cmap="autumn");

In [ ]:
#Intialize model with linkage = single
agg = AgglomerativeClustering(n_clusters=2, linkage="single")
#Fit on circles dataset
agg.fit(moons)
labels = agg.labels_

plt.figure(figsize=(10, 8))
plt.scatter(x = moons[:, 0], y = moons[:, 1], c=labels, cmap="autumn");

In [ ]:
#Intialize model with linkage = complete
agg = AgglomerativeClustering(n_clusters=2, linkage="average")
#Fit on circles dataset
agg.fit(moons)
labels = agg.labels_

plt.figure(figsize=(10, 8))
plt.scatter(x = moons[:, 0], y = moons[:, 1], c=labels, cmap="autumn")

### Spotify Data

Now let's try it out on the Spotify data to see if it offers an improvement over K-Means.

Like we did with K-Means, we're going to iterate over a range of k values and then determine the best k using silhouette scores.

In [ ]:
#Intialize list to collect silhouette scores
s_scores = []

#iterate over kvalues
for k in kvalues:
    #intialize model with k and fit it on Xs; use affinity="euclidean" on scikit v<1.2
    agg = AgglomerativeClustering(n_clusters=k, metric="euclidean", linkage="ward", memory="..")
    agg.fit(Xs)
    labels = agg.labels_
    #derive silhouette score by passing in data and labels
    ss = silhouette_score(Xs, labels=labels)
    #append silhouette score to s_scores
    s_scores.append(ss)
    
#plot kvalues versus s_scores
plt.figure(figsize=(10, 8))
plt.plot(kvalues, s_scores, linewidth = 3)
plt.xticks(ticks=kvalues, labels=kvalues)
plt.xlabel("K Values")
plt.ylabel("Silhouette Scores");

Now five clusters seems to be ideal! Let's look at the results.

In [ ]:
# use affinity="euclidean" on scikit v<1.2
agg = AgglomerativeClustering(n_clusters=5, metric="euclidean", linkage="ward", memory="..")
agg.fit(Xs)
labels = agg.labels_

pd.value_counts(labels)
X["labels2"] = labels
X.groupby('labels2').mean()

In [ ]:
X.groupby('labels2')['labels'].describe().T.round(3)

**Question**: How closely related do the two sets of labels appear to be?